## Importar Bibliotecas

In [35]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols
import numpy as np
import matplotlib.pyplot as plt

### Funções Auxiliares

In [36]:
def detect_treatment_horizon(df, treatment_var, unit_var, time_var):
    """
    Detects the horizon (time of treatment start) for each unit.
    
    Parameters:
    df (DataFrame): The dataset with units, time, and treatment indicator.
    treatment_var (str): The column name indicating treatment status (0/1).
    unit_var (str): The column name identifying the units.
    time_var (str): The column name indicating the time period.
    
    Returns:
    DataFrame: A DataFrame with an additional column 'treatment_horizon', which 
               contains the time when treatment starts for each unit, or NaN if not treated.
    """
    df['treatment_horizon'] = df.groupby(unit_var)[treatment_var].transform(lambda x: (x == 1).idxmax() if x.sum() > 0 else np.nan)
    return df


def apply_dynamic_clean_control(df, treatment_var, time_var, treatment_horizon_var):
    """
    Applies clean control dynamically by excluding units that are treated at or before each time horizon.
    
    Parameters:
    df (DataFrame): Panel data with treatment indicator and treatment horizon.
    treatment_var (str): Treatment variable name.
    time_var (str): Time variable name.
    treatment_horizon_var (str): Column name indicating the treatment start time (horizon) for each unit.
    
    Returns:
    DataFrame: A filtered DataFrame where treated units are excluded from control after their treatment start time.
    """
    df_clean = df[(df[time_var] < df[treatment_horizon_var]) | df[treatment_horizon_var].isna()]
    return df_clean


def local_projection_full_treatment(df, outcome_var, treatment_var, unit_var, time_var, treatment_horizon_var, covariates=None):
    """
    Applies Local Projections method with dynamic horizons that span the entire treatment period.
    
    Parameters:
    df (DataFrame): Panel data with treatment, outcome, covariates, and treatment horizon.
    outcome_var (str): Outcome variable name.
    treatment_var (str): Treatment variable name.
    unit_var (str): Unit identifier column.
    time_var (str): Time variable name.
    treatment_horizon_var (str): Column name with treatment horizon (start time).
    covariates (list): List of covariate variable names (optional).
    
    Returns:
    results (dict): Dictionary with regression results for each horizon.
    """
    # Get the maximum time period to calculate remaining periods after treatment start
    max_time = df[time_var].max()

    # Initialize results dictionary
    results = {}

    # Loop over each unit and calculate dynamic horizons
    for unit in df[unit_var].unique():
        # Filter the data for the current unit
        df_unit = df[df[unit_var] == unit]

        # Identify the treatment start time (horizon)
        treatment_start = df_unit[treatment_horizon_var].iloc[0]

        # Define dynamic horizons based on remaining periods after treatment start
        if not pd.isna(treatment_start):
            remaining_horizons = list(range(0, int(max_time - treatment_start + 1)))
        else:
            continue  # If no treatment for this unit, skip

        # Loop over each horizon and estimate the treatment effect
        for h in remaining_horizons:
            df[f'outcome_h{h}'] = df.groupby(unit_var)[outcome_var].shift(-h) - df[outcome_var]
            df_h = df.dropna(subset=[f'outcome_h{h}', treatment_var])

            # Apply dynamic clean control for this horizon
            df_h = apply_dynamic_clean_control(df_h, treatment_var, time_var, treatment_horizon_var)

            # Create the regression formula
            formula = f"outcome_h{h} ~ {treatment_var}"
            if covariates:
                formula += ' + ' + ' + '.join(covariates)

            # Run the regression for this horizon
            model = ols(formula, data=df_h).fit(cov_type='cluster', cov_kwds={'groups': df_h[unit_var]})

            # Store the result
            results[h] = model
            print(f"Horizon {h} for unit {unit}:\n", model.summary())

    return results


# Function to Calculate ATT
def calculate_att(results):
    """
    Calculate the Average Treatment Effect on the Treated (ATT) across horizons.
    
    Parameters:
    results (dict): Dictionary of regression results from local projection.

    Returns:
    att (float): The weighted average treatment effect across horizons.
    """
    att = np.mean([result.params['treatment'] for h, result in results.items()])
    return att

# Function to Plot Results
def plot_results(results):
    """
    Plots the treatment effects across time horizons.

    Parameters:
    results (dict): Dictionary of regression results from local projection.
    """
    horizons = list(results.keys())
    estimates = [result.params['treatment'] for result in results.values()]
    conf_ints = [result.conf_int().loc['treatment'] for result in results.values()]

    plt.errorbar(horizons, estimates, 
                 yerr=[(ci[1] - ci[0]) / 2 for ci in conf_ints], 
                 fmt='o-', label='Treatment Effect')
    plt.axhline(0, color='black', linestyle='--')
    plt.xlabel('Horizon')
    plt.ylabel('Treatment Effect')
    plt.title('Local Projections: Treatment Effects Over Time')
    plt.legend()
    plt.show()

### Geração Dado Aleatório Tratamento Escalonado Efeito Homogêneo e Dinâmico

In [37]:
import numpy as np
import pandas as pd
from statsmodels.tsa.holtwinters import ExponentialSmoothing

# Definir parâmetros para o dataset
n_units = 9  # número de unidades (indivíduos)
n_periods = 36  # número de períodos de tempo
proportion_treated = 1/3  # proporção de indivíduos que recebem o tratamento
treatment_effect = 0.09  # efeito homogêneo do tratamento
random_walk_effect = 0.252  # intensidade do efeito do random walk

# Definir o ponto de início do tratamento (com adoção escalonada, ou seja, "staggered adoption")
min_treatment_start = int(0.5 * n_periods)  # Tratamento pode começar após 20% do tempo
max_treatment_start = int(0.8 * n_periods)  # Tratamento não pode começar após 80% do tempo

# Gerar IDs para as unidades
ids = np.repeat(np.arange(n_units), n_periods)

# Gerar a variável de tempo
time = np.tile(np.arange(n_periods), n_units)

# Selecionar 1/3 dos indivíduos para o tratamento
treated_units = np.random.choice(np.arange(n_units), size=int(n_units * proportion_treated), replace=False)

# Gerar o tratamento (tratamento começa em momentos escalonados, entre 20% e 80% do tempo)
treatment_start = np.random.choice(np.arange(min_treatment_start, max_treatment_start), size=n_units)
treatment = np.zeros(n_units * n_periods)

# Aplicar tratamento escalonado para os indivíduos selecionados
for i in treated_units:
    treatment[i * n_periods + treatment_start[i]: (i + 1) * n_periods] = 1

# Função para gerar séries temporais utilizando Holt-Winters (com sazonalidade)
def generate_holt_winters_series(n_periods, seasonal_periods=12):
    # Gera dados com sazonalidade
    time = np.arange(n_periods)
    seasonal_effect = np.sin(2 * np.pi * time / seasonal_periods)
    trend = 0.05 * time
    noise = np.random.normal(0, 0.1, n_periods)
    
    # Dados base (com tendência, sazonalidade e ruído)
    data = trend + seasonal_effect + noise
    
    # Aplicar Holt-Winters para suavizar a série
    model = ExponentialSmoothing(data, trend="add", seasonal="add", seasonal_periods=seasonal_periods)
    fit = model.fit(optimized=True)
    return fit.fittedvalues

# Função para gerar um pequeno random walk
def generate_random_walk(n_periods, step_size=0.02):
    steps = np.random.normal(0, step_size, n_periods)
    return np.cumsum(steps)  # Caminhada aleatória com soma cumulativa

# Inicializar a lista para armazenar as séries temporais simuladas
y_values = []

# Gerar a série temporal para cada indivíduo
for i in range(n_units):
    # Gerar a série temporal usando Holt-Winters com sazonalidade
    y_series = generate_holt_winters_series(n_periods)
    
    # Adicionar um pequeno random walk à série temporal
    random_walk = generate_random_walk(n_periods, step_size=random_walk_effect)
    y_series += random_walk
    
    # Se o indivíduo for tratado, aplicar o efeito dinâmico e homogêneo do tratamento
    if i in treated_units:
        treatment_period_start = treatment_start[i]
        
        # Efeito dinâmico: O efeito do tratamento aumenta ao longo do tempo, mas é homogêneo entre unidades
        for t in range(treatment_period_start, n_periods):
            dynamic_effect = treatment_effect * (t - treatment_period_start) / (n_periods - treatment_period_start)  # Efeito dinâmico ajustado
            y_series[t] += dynamic_effect  # Efeito cumulativo, mas homogêneo entre unidades
    
    # Armazenar a série temporal gerada
    y_values.extend(y_series)

# Gerar covariáveis aleatórias
covariate1 = np.random.normal(size=n_units * n_periods)
covariate2 = np.random.normal(size=n_units * n_periods)

# Criar o DataFrame final
homogeneo = pd.DataFrame({
    'id': ids,
    'time': time,
    'treatment': treatment,
    'y': y_values,
    'covariate1': covariate1,
    'covariate2': covariate2
})

# Exibir as primeiras linhas do dataset gerado
print(homogeneo.head())

# Salvar em um arquivo CSV, se necessário
# homogeneo.to_csv('synthetic_data_homogeneous.csv', index=False)



   id  time  treatment         y  covariate1  covariate2
0   0     0        0.0  0.080252    0.157854    0.518754
1   0     1        0.0  1.049027   -0.550852    0.500852
2   0     2        0.0  1.658637   -2.943036   -1.429276
3   0     3        0.0  1.911825    0.129450    0.240353
4   0     4        0.0  1.922124   -3.043765   -0.342243


### Geração Dado Aleatório Tratamento Escalonado Efeito Heterogeneo e Dinâmico

In [38]:
import numpy as np
import pandas as pd
from statsmodels.tsa.holtwinters import ExponentialSmoothing

# Definir parâmetros para o dataset
n_units = 9  # número de unidades (indivíduos)
n_periods = 36  # número de períodos de tempo
proportion_treated = 1/3  # proporção de indivíduos que recebem o tratamento
random_walk_effect = 0.42  # intensidade do efeito do random walk

# Definir o ponto de início do tratamento (com adoção escalonada, ou seja, "staggered adoption")
min_treatment_start = int(0.5 * n_periods)  # Tratamento pode começar após 20% do tempo
max_treatment_start = int(0.8 * n_periods)  # Tratamento não pode começar após 80% do tempo

# Gerar IDs para as unidades
ids = np.repeat(np.arange(n_units), n_periods)

# Gerar a variável de tempo
time = np.tile(np.arange(n_periods), n_units)

# Selecionar 1/3 dos indivíduos para o tratamento
treated_units = np.random.choice(np.arange(n_units), size=int(n_units * proportion_treated), replace=False)

# Gerar o tratamento (tratamento começa em momentos escalonados, entre 20% e 80% do tempo)
treatment_start = np.random.choice(np.arange(min_treatment_start, max_treatment_start), size=n_units)
treatment = np.zeros(n_units * n_periods)

# Aplicar tratamento escalonado para os indivíduos selecionados
for i in treated_units:
    treatment[i * n_periods + treatment_start[i]: (i + 1) * n_periods] = 1

# Função para gerar séries temporais utilizando Holt-Winters (com sazonalidade)
def generate_holt_winters_series(n_periods, seasonal_periods=12):
    # Gera dados com sazonalidade
    time = np.arange(n_periods)
    seasonal_effect = np.sin(2 * np.pi * time / seasonal_periods)
    trend = 0.05 * time
    noise = np.random.normal(0, 0.1, n_periods)
    
    # Dados base (com tendência, sazonalidade e ruído)
    data = trend + seasonal_effect + noise
    
    # Aplicar Holt-Winters para suavizar a série
    model = ExponentialSmoothing(data, trend="add", seasonal="add", seasonal_periods=seasonal_periods)
    fit = model.fit(optimized=True)
    return fit.fittedvalues

# Função para gerar um pequeno random walk
def generate_random_walk(n_periods, step_size=0.02):
    steps = np.random.normal(0, step_size, n_periods)
    return np.cumsum(steps)  # Caminhada aleatória com soma cumulativa

# Inicializar a lista para armazenar as séries temporais simuladas
y_values = []

# Gerar uma intensidade de efeito de tratamento diferente para cada unidade tratada (heterogêneo)
treatment_effects = {i: np.random.uniform(0.03, 0.08) for i in treated_units}  # Efeito heterogêneo

# Gerar a série temporal para cada indivíduo
for i in range(n_units):
    # Gerar a série temporal usando Holt-Winters com sazonalidade
    y_series = generate_holt_winters_series(n_periods)
    
    # Adicionar um pequeno random walk à série temporal
    random_walk = generate_random_walk(n_periods, step_size=random_walk_effect)
    y_series += random_walk
    
    # Se o indivíduo for tratado, aplicar o efeito dinâmico e heterogêneo do tratamento
    if i in treated_units:
        treatment_period_start = treatment_start[i]
        treatment_effect = treatment_effects[i]  # Efeito heterogêneo para cada indivíduo
        
        # Efeito dinâmico: O efeito do tratamento aumenta ao longo do tempo, mas varia entre unidades
        for t in range(treatment_period_start, n_periods):
            dynamic_effect = treatment_effect * (t - treatment_period_start) / (n_periods - treatment_period_start)  # Efeito dinâmico ajustado
            y_series[t] += dynamic_effect  # Efeito cumulativo e heterogêneo entre unidades
    
    # Armazenar a série temporal gerada
    y_values.extend(y_series)

# Gerar covariáveis aleatórias
covariate1 = np.random.normal(size=n_units * n_periods)
covariate2 = np.random.normal(size=n_units * n_periods)

# Criar o DataFrame final
hetero = pd.DataFrame({
    'id': ids,
    'time': time,
    'treatment': treatment,
    'y': y_values,
    'covariate1': covariate1,
    'covariate2': covariate2
})

# Exibir as primeiras linhas do dataset gerado
print(hetero.head())

# Salvar em um arquivo CSV, se necessário
hetero.to_csv('synthetic_data_heterogeneous.csv', index=False)
hetero.to_excel('synthetic_data_heterogeneous.xlsx', index=False)

   id  time  treatment         y  covariate1  covariate2
0   0     0        0.0  0.126878    1.901688   -0.765481
1   0     1        0.0  0.596166   -0.787539    0.054296
2   0     2        0.0  0.232557   -0.196677   -1.105477
3   0     3        0.0  0.578233    0.713591    0.496531
4   0     4        0.0  0.421588   -0.379350    0.273275


### Aplicando na simulação

In [42]:
hetero = pd.read_excel("H:\\Meu Drive\\Matheus Andrade\\CASA CIVIL\\Analises\\gsynth_pacto\\turnout.xlsx")
hetero

,abb,year,turnout,policy_edr,policy_mail_in,policy_motor
0,AL,1920,21.021074,0,0,0
1,AL,1924,13.582329,0,0,0
2,AL,1928,19.038504,0,0,0
3,AL,1932,17.620020,0,0,0
4,AL,1936,18.692375,0,0,0
...,...,...,...,...,...,...
1123,WY,1996,61.117554,1,0,0
1124,WY,2000,58.569672,1,0,0
1125,WY,2004,63.700001,1,0,0
1126,WY,2008,60.900002,1,0,0


In [39]:
# Apply Clean Control
hetero_clean = apply_clean_control(hetero, 'treatment', 'year', horizon=1)

# Define time horizons
horizons = range(0,20)

# Run Local Projections
results = local_projection(hetero_clean, 'y', 'treatment', horizons, 'id', 'time', covariates=['covariate1', 'covariate2'])

# Calculate ATT
att = calculate_att(results)
print(f"Average Treatment Effect on the Treated (ATT): {att}")

# Plot Results
plot_results(results)

KeyError: 'treatment'